In [12]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from tqdm import tqdm
import pubchempy as pcp
import requests

In [13]:
pharmgkb_df = pd.read_csv('rawData/PharmGKB/clinicalVariants/clinicalVariants.tsv', sep='\t')
# print(pharmgkb_df)
pharmgkb_df.head()

,variant,gene,type,level of evidence,chemicals,phenotypes
0,"CYP2C9*1, CYP2C9*2, CYP2C9*3, CYP2C9*13",CYP2C9,Metabolism/PK,1A,meloxicam,NaN
1,"CYP2C9*1, CYP2C9*3, CYP2C9*13",CYP2C9,Metabolism/PK,1A,lornoxicam,NaN
2,"CYP2C9*1, CYP2C9*2, CYP2C9*3",CYP2C9,Metabolism/PK,1A,siponimod,NaN
3,rs17376848,DPYD,Toxicity,1A,capecitabine,Neoplasms
4,rs2297595,DPYD,Toxicity,1A,capecitabine,Neoplasms


In [14]:
chemicals_df = pd.read_csv('rawData/PharmGKB/chemicals/chemicals.tsv', sep='\t')
chemicals_df['PubChem Compound Identifiers'] = chemicals_df['PubChem Compound Identifiers'].astype(object)
chemicals_df.head()


,PharmGKB Accession Id,Name,Generic Names,Trade Names,Brand Mixtures,Type,Cross-references,SMILES,InChI,Dosing Guideline,...,VIP Count,Dosing Guideline Sources,Top Clinical Annotation Level,Top FDA Label Testing Level,Top Any Drug Label Testing Level,Label Has Dosing Info,Has Rx Annotation,RxNorm Identifiers,ATC Identifiers,PubChem Compound Identifiers
0,PA166250381,10-desmethyl alpha-dihydrotetrabenazine,"""(2R,3R,11bR)-9-methoxy-3-(2-methylpropyl)-2,3...",NaN,NaN,Metabolite,PubChem Compound:59272813,NaN,NaN,No,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.92728e+07
1,PA166250405,10-desmethyl beta-dihydrodeutetrabenazine,"""(2S,3R,11bR)-3-(2-methylpropyl)-9-(trideuteri...",NaN,NaN,Metabolite,PubChem Compound:117693284,NaN,NaN,No,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.17693e+08
2,PA166250403,10-desmethyl beta-dihydrotetrabenazine,"""(2S,3R,11bR)-9-methoxy-3-(2-methylpropyl)-2,3...",NaN,NaN,Metabolite,PubChem Compound:59272796,NaN,NaN,No,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.92728e+07
3,PA166178620,10-formyltetrahydrofolate,NaN,NaN,NaN,Biological Intermediate,PubChem Compound:10,NaN,NaN,No,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10
4,PA166131395,10-hydroxy r-warfarin,NaN,NaN,NaN,Metabolite,HMDB:HMDB13884,NaN,NaN,No,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
pharmgkb_all_df = pd.DataFrame()
for i in range(len(pharmgkb_df)):
    chemicals = pharmgkb_df['chemicals'][i]
    variant = pharmgkb_df['variant'][i]
    gene_symbol = pharmgkb_df['gene'][i]
    label = pharmgkb_df['type'][i]
    disease_name = pharmgkb_df['phenotypes'][i]

    try:
        chemical_type = chemicals_df[chemicals_df['Name'] == chemicals]['Type'].values[0]
        chemical_smile = chemicals_df[chemicals_df['Name'] == chemicals]['SMILES'].values[0]
        pubchem_id = chemicals_df[chemicals_df['Name'] == chemicals]['PubChem Compound Identifiers'].values[0]
        # print(chemicals_df[chemicals_df['Name'] == chemicals]['PubChem Compound Identifiers'].values[0])
    except IndexError:
        # print(chemicals)
        continue
    # except KeyError:
    #     print(chemicals)
    #     continue

    pharmgkb_all_df = pharmgkb_all_df.append([{'data source': 'pharmgkb', 'drug name': chemicals, 'pubchem id': pubchem_id, 'smile': chemical_smile, 'gene symbol': gene_symbol, 'variant': variant, 'label': label}], ignore_index = True)

pharmgkb_all_df.to_csv('middlefile/pharmgkb_all_table.tsv')
pharmgkb_all_df

,data source,drug name,pubchem id,smile,gene symbol,variant,label
0,pharmgkb,meloxicam,5281106.0,CC1=CN=C(S1)N/C(=C\2/C(=O)C3=CC=CC=C3S(=O)(=O)...,CYP2C9,"CYP2C9*1, CYP2C9*2, CYP2C9*3, CYP2C9*13",Metabolism/PK
1,pharmgkb,lornoxicam,5282204.0,CN1/C(=C(\NC2=CC=CC=N2)/O)/C(=O)C3=C(S1(=O)=O)...,CYP2C9,"CYP2C9*1, CYP2C9*3, CYP2C9*13",Metabolism/PK
2,pharmgkb,siponimod,44599207.0,NaN,CYP2C9,"CYP2C9*1, CYP2C9*2, CYP2C9*3",Metabolism/PK
3,pharmgkb,capecitabine,60953.0,CCCCCOC(=O)NC1=NC(=O)N(C=C1F)[C@H]2[C@@H]([C@@...,DPYD,rs17376848,Toxicity
4,pharmgkb,capecitabine,60953.0,CCCCCOC(=O)NC1=NC(=O)N(C=C1F)[C@H]2[C@@H]([C@@...,DPYD,rs2297595,Toxicity
...,...,...,...,...,...,...,...
4152,pharmgkb,sertraline,68617.0,CN[C@H]1CC[C@H](C2=CC=CC=C12)C3=CC(=C(C=C3)Cl)Cl,SLC6A4,"SLC6A4 HTTLPR long form (L allele), SLC6A4 HTT...",Efficacy
4153,pharmgkb,sertraline,68617.0,CN[C@H]1CC[C@H](C2=CC=CC=C12)C3=CC(=C(C=C3)Cl)Cl,SLC6A4,"SLC6A4 HTTLPR long form (L allele), SLC6A4 HTT...",Toxicity
4154,pharmgkb,fluoxetine,3386.0,CNCCC(C1=CC=CC=C1)OC2=CC=C(C=C2)C(F)(F)F,SLC6A4,rs25531,Efficacy
4155,pharmgkb,fluoxetine,3386.0,CNCCC(C1=CC=CC=C1)OC2=CC=C(C=C2)C(F)(F)F,SLC6A4,"SLC6A4 HTTLPR long form (L allele), SLC6A4 HTT...",Efficacy
